# BBC News Article classification

**Business Problem :** Create a classification model which classifiy the article into category

**Dataset location :**[Kaggle](https://www.kaggle.com/c/learn-ai-bbc/data)

**Description from Kaggle :**

Text documents are one of the richest sources of data for businesses.

We’ll use a public dataset from the BBC comprised of 2225 articles, each labeled under one of 5 categories: business, entertainment, politics, sport or tech.

The dataset is broken into 1490 records for training and 735 for testing. The goal will be to build a system that can accurately classify previously unseen news articles into the right category

**Dataset Description:**

ArticleId - Article id unique # given to the record
Article - text of the header and article
Category - cateogry of the article (tech, business, sport, entertainment, politics/li>

Lets get started,
As this is the text classification NLP problem , we are going to solve this problem with Deep learning library called as Keras

In [48]:
# Import libraries
import pandas as pd
import numpy as np
import re
import collections
import matplotlib.pyplot as plt
from pathlib import Path
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from keras import models
from keras import layers
from tensorflow.python import keras
import keras
from keras.initializers import Constant
from keras.layers import LSTM
import nltk
#nltk.download()
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
import pandas_profiling as pp

### Load dataset

In [49]:
data = pd.read_csv('bbc-text.csv')
data.head()

,category,text
0,tech,tv future in the hands of viewers with home th...
1,business,worldcom boss left books alone former worldc...
2,sport,tigers wary of farrell gamble leicester say ...
3,sport,yeading face newcastle in fa cup premiership s...
4,entertainment,ocean s twelve raids box office ocean s twelve...


## Exploratary Data Analysis

1. Now days, we have library called pandas-profiliing which makes our life more easier for data analysis
2. pandas profilling give all analysis which require for understanding the data.
3. from profilling , we got information like, there is no missing data in dataset, also got target variable distributon and count
4. its look like 99 rows are duplicate in our dataset , lets remove those rows
5. take a copy of dataset and start working on it

In [50]:
pp.ProfileReport(data)

Number of variables,2
Number of observations,2225
Total Missing (%),0.0%
Total size in memory,34.8 KiB
Average record size in memory,16.0 B
Numeric,0
Categorical,2
Boolean,0
Date,0
Text (Unique),0
Rejected,0


In [51]:
df = data.copy()

In [52]:
%time df.drop_duplicates(subset ="text",keep=False,inplace=True)
df.shape

Wall time: 15 ms


(2027, 2)

In [53]:
df = df.reset_index(drop=True)

In [54]:
df['category'].value_counts()

sport            497
business         496
politics         389
entertainment    352
tech             293
Name: category, dtype: int64

lets look at first article of news and understand to make it clean for next procedure what we need to do.

In [55]:
df['text'][0]

'tv future in the hands of viewers with home theatre systems  plasma high-definition tvs  and digital video recorders moving into the living room  the way people watch tv will be radically different in five years  time.  that is according to an expert panel which gathered at the annual consumer electronics show in las vegas to discuss how these new technologies will impact one of our favourite pastimes. with the us leading the trend  programmes and other content will be delivered to viewers via home networks  through cable  satellite  telecoms companies  and broadband service providers to front rooms and portable devices.  one of the most talked-about technologies of ces has been digital and personal video recorders (dvr and pvr). these set-top boxes  like the us s tivo and the uk s sky+ system  allow people to record  store  play  pause and forward wind tv programmes when they want.  essentially  the technology allows for much more personalised tv. they are also being built-in to high

## Removing stopwords

In [56]:
## Text cleaning
## Remove stopwords
def stopword(data):
    
    stopwordlist= stopwords.words('english')
    
    data = data.split()
    clean_data = [word for word in data if (word not in stopwordlist)]
    return " ".join(clean_data)    

In [57]:
df.text = df.text.apply(stopword)

In [58]:
df.text[0]

'tv future hands viewers home theatre systems plasma high-definition tvs digital video recorders moving living room way people watch tv radically different five years time. according expert panel gathered annual consumer electronics show las vegas discuss new technologies impact one favourite pastimes. us leading trend programmes content delivered viewers via home networks cable satellite telecoms companies broadband service providers front rooms portable devices. one talked-about technologies ces digital personal video recorders (dvr pvr). set-top boxes like us tivo uk sky+ system allow people record store play pause forward wind tv programmes want. essentially technology allows much personalised tv. also built-in high-definition tv sets big business japan us slower take europe lack high-definition programming. people forward wind adverts also forget abiding network channel schedules putting together a-la-carte entertainment. us networks cable satellite companies worried means terms a

## Using textblob work on root of the word with help of lemmatization

In [59]:
### lemmitization
from textblob import Word
def lem_apply(data):
    
    data=data.split()
    new_data = [Word(word).lemmatize()  for word in data]
    return " ".join(new_data)

In [60]:
df.text = df.text.apply(lem_apply)

In [61]:
df.text[0]

'tv future hand viewer home theatre system plasma high-definition tv digital video recorder moving living room way people watch tv radically different five year time. according expert panel gathered annual consumer electronics show la vega discus new technology impact one favourite pastimes. u leading trend programme content delivered viewer via home network cable satellite telecom company broadband service provider front room portable devices. one talked-about technology ce digital personal video recorder (dvr pvr). set-top box like u tivo uk sky+ system allow people record store play pause forward wind tv programme want. essentially technology allows much personalised tv. also built-in high-definition tv set big business japan u slower take europe lack high-definition programming. people forward wind advert also forget abiding network channel schedule putting together a-la-carte entertainment. u network cable satellite company worried mean term advertising revenue well brand identity

## dataset splitting for training & Testing

In [62]:
X_train, X_test, Y_train, Y_test = train_test_split(df.text, df.category, test_size=0.2, random_state=37)

## Tokenizer and filtering with keras

1. out of the all words we are taking on 10000 words
2. with keras API of Tokenizer which tokenize the sentence also do data cleaning and split the data

In [63]:
size_words=10000
ks_tok = Tokenizer(num_words=size_words,filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n0123456789',lower=True,split=" ")
ks_tok.fit_on_texts(X_train)

## converting the word into sequence of number

In [64]:
X_train_seq = ks_tok.texts_to_sequences(X_train) # convert each sentence into number 
X_test_seq = ks_tok.texts_to_sequences(X_test)

In [65]:
max_count = X_train.apply(lambda x: len(x.split(' ')))
max_count.describe()

count    1621.000000
mean      225.090685
std       137.161430
min        49.000000
25%       144.000000
50%       196.000000
75%       275.000000
max      2278.000000
Name: text, dtype: float64

## Padding
Adding zero to each sentence and making it equal to sentence of maximum lenght 

In [66]:
X_train_seq_trunc = pad_sequences(X_train_seq, maxlen=2278)
X_test_seq_trunc = pad_sequences(X_test_seq, maxlen=2278)
X_test_seq_trunc[0]

array([   0,    0,    0, ..., 1545,  424,  137])

## Encoding the target feature in number 

In [67]:
le = LabelEncoder()
y_train_le = le.fit_transform(Y_train)
y_test_le = le.transform(Y_test)
y_train_oh = to_categorical(y_train_le)
y_test_oh = to_categorical(y_test_le)

## Neural Network with Keras
1. we are using functional API of keras to build model
2. Initially we add input layer which take input with size of max 2278
3. then we add embedding layer of max word which we have earlier decided 
4. use dropout and flattern to make input sequencially
5. output layer with dense 5 which is equal to number of unique target feature and will use softmax as activation function
6. then, we have to compile our model and below is the overall summary of model.

In [45]:
deep_inputs = layers.Input(shape=(2278,))
embedding= layers.Embedding(10000 , 8 ,input_length=2278 )(deep_inputs)
embedding = layers.Dropout(0.5)(embedding)
#embedding = layers.Dense(100, activation='sigmoid')(embedding)
embedding = layers.Flatten()(embedding)
embedding_out = layers.Dense(5, activation='softmax')(embedding)
deep_model = keras.Model(inputs=deep_inputs,outputs=embedding_out)
deep_model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
deep_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 2278)              0         
_________________________________________________________________
embedding_2 (Embedding)      (None, 2278, 8)           80000     
_________________________________________________________________
dropout_2 (Dropout)          (None, 2278, 8)           0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 18224)             0         
_________________________________________________________________
dense_2 (Dense)              (None, 5)                 91125     
Total params: 171,125
Trainable params: 171,125
Non-trainable params: 0
_________________________________________________________________


## Train the model on training data 
we can change epoch and batch_size according to our requirement , we also need to check model performance on different value of epoch and batch_size

In [46]:
deep_model.fit(X_train_seq_trunc,y_train_oh,epochs=50,batch_size=512)

Epoch 1/50
1621/1621 [==============================] - 2s 1ms/step - loss: 1.7642 - acc: 0.2054
Epoch 2/50
1621/1621 [==============================] - 1s 788us/step - loss: 1.5620 - acc: 0.2893
Epoch 3/50
1621/1621 [==============================] - 1s 767us/step - loss: 1.5200 - acc: 0.3171
Epoch 4/50
1621/1621 [==============================] - 1s 769us/step - loss: 1.4943 - acc: 0.4078
Epoch 5/50
1621/1621 [==============================] - 1s 711us/step - loss: 1.4374 - acc: 0.3701
Epoch 6/50
1621/1621 [==============================] - 1s 628us/step - loss: 1.3865 - acc: 0.4701
Epoch 7/50
1621/1621 [==============================] - 1s 728us/step - loss: 1.3200 - acc: 0.5713
Epoch 8/50
1621/1621 [==============================] - 1s 680us/step - loss: 1.2801 - acc: 0.4898
Epoch 9/50
1621/1621 [==============================] - 1s 734us/step - loss: 1.2300 - acc: 0.5460
Epoch 10/50
1621/1621 [==============================] - 1s 788us/step - loss: 1.1750 - acc: 0.5830
Epoch 11/50

## Model evaluation on training data

In [47]:
result=deep_model.evaluate(X_train_seq_trunc,y_train_oh)
result

1621/1621 [==============================] - 0s 203us/step


[0.031847974345235085, 1.0]

## Model evaluation on Testing data

In [48]:
result=deep_model.evaluate(X_test_seq_trunc,y_test_oh)
result

406/406 [==============================] - 0s 61us/step


[0.13823234855101027, 0.9704433497536946]

## Model building with CNN with keras

In [49]:

deep_inputs = layers.Input(shape=(2278,))
embedding= layers.Embedding(10000 , 8 ,input_length=2278 )(deep_inputs)
embedding = layers.Conv1D(256, 5, activation='relu')(embedding)
embedding = layers.GlobalMaxPooling1D()(embedding)
#embedding = layers.Dense(128, activation='relu')(embedding)
#embedding = layers.Dropout(0.5)(embedding)
#embedding = layers.Flatten()(embedding)
embedding_out = layers.Dense(5, activation='softmax')(embedding)
deep_model_CNN = keras.Model(inputs=deep_inputs,outputs=embedding_out)
deep_model_CNN.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
deep_model_CNN.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 2278)              0         
_________________________________________________________________
embedding_3 (Embedding)      (None, 2278, 8)           80000     
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 2274, 256)         10496     
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 256)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 5)                 1285      
Total params: 91,781
Trainable params: 91,781
Non-trainable params: 0
_________________________________________________________________


## Train the model on training data CNN

In [50]:
deep_model_CNN.fit(X_train_seq_trunc,y_train_oh,epochs=50,batch_size=512)

Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Epoch 1/50
1621/1621 [==============================] - 34s 21ms/step - loss: 1.6016 - acc: 0.2517
Epoch 2/50
1621/1621 [==============================] - 32s 20ms/step - loss: 1.5828 - acc: 0.2517
Epoch 3/50
1621/1621 [==============================] - 31s 19ms/step - loss: 1.5719 - acc: 0.2893
Epoch 4/50
1621/1621 [==============================] - 30s 18ms/step - loss: 1.5626 - acc: 0.2647
Epoch 5/50
1621/1621 [==============================] - 31s 19ms/step - loss: 1.5511 - acc: 0.2566
Epoch 6/50
1621/1621 [==============================] - 28s 18ms/step - loss: 1.5354 - acc: 0.3961
Epoch 7/50
1621/1621 [==============================] - 28s 17ms/step - loss: 1.5147 - acc: 0.4534
Epoch 8/50
1621/1621 [==============================] - 28s 17ms/step - loss: 1.4882 - acc: 0.4775
Epoch 9/50
1621/1621 [==============================] - 31s 19ms/step - loss: 1.4555 - acc: 0.4911
Epoch 10/50
1621/1621 [========

## Model evaluation on training data CNN

In [51]:
result=deep_model_CNN.evaluate(X_train_seq_trunc,y_train_oh)
result

1621/1621 [==============================] - 3s 2ms/step


[0.024931464101808958, 0.9987661937075879]

## Model evaluation on training data CNN

In [52]:
result=deep_model_CNN.evaluate(X_test_seq_trunc,y_test_oh)
result

406/406 [==============================] - 1s 2ms/step


[0.1439388860475841, 0.9581280779368772]

## Model building using GRU

In [29]:
deep_inputs = layers.Input(shape=(2278,))
embedding= layers.Embedding(10000 , 8 ,input_length=2278 )(deep_inputs)
embedding = layers.GRU(100,dropout=0.5, recurrent_dropout=0.5)(embedding)
#embedding = layers.Dense(256, activation='sigmoid')(embedding)
#embedding = layers.Dropout(0.5)(embedding)
#embedding = layers.Flatten()(embedding)
embedding_out = layers.Dense(5, activation='softmax')(embedding)
deep_model_gru = keras.Model(inputs=deep_inputs,outputs=embedding_out)
deep_model_gru.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
deep_model_gru.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 2278)              0         
_________________________________________________________________
embedding_3 (Embedding)      (None, 2278, 8)           80000     
_________________________________________________________________
gru_1 (GRU)                  (None, 100)               32700     
_________________________________________________________________
dense_3 (Dense)              (None, 5)                 505       
Total params: 113,205
Trainable params: 113,205
Non-trainable params: 0
_________________________________________________________________


In [30]:
deep_model_gru.fit(X_train_seq_trunc,y_train_oh,epochs=50,batch_size=512)

Epoch 1/50
1780/1780 [==============================] - 20s 11ms/step - loss: 1.6074 - acc: 0.2090
Epoch 2/50
1780/1780 [==============================] - 19s 11ms/step - loss: 1.6019 - acc: 0.2354
Epoch 3/50
1780/1780 [==============================] - 20s 11ms/step - loss: 1.5968 - acc: 0.2320
Epoch 4/50
1780/1780 [==============================] - 22s 12ms/step - loss: 1.5912 - acc: 0.2478
Epoch 5/50
1780/1780 [==============================] - 20s 11ms/step - loss: 1.5817 - acc: 0.2365
Epoch 6/50
1780/1780 [==============================] - 20s 11ms/step - loss: 1.5673 - acc: 0.2545
Epoch 7/50
1780/1780 [==============================] - 20s 11ms/step - loss: 1.5465 - acc: 0.3051
Epoch 8/50
1780/1780 [==============================] - 20s 11ms/step - loss: 1.5182 - acc: 0.3433
Epoch 9/50
1780/1780 [==============================] - 21s 12ms/step - loss: 1.4849 - acc: 0.4612
Epoch 10/50
1780/1780 [==============================] - 21s 12ms/step - loss: 1.4568 - acc: 0.4596
Epoch 11/

## model evaluation on testing data using GRU

In [31]:
result=deep_model_gru.evaluate(X_test_seq_trunc,y_test_oh)
result

445/445 [==============================] - 5s 11ms/step


[0.7343408161334777, 0.7685393275839559]

In [69]:
## LSTM 
deep_inputs = layers.Input(shape=(2278,))
embedding= layers.Embedding(10000 , 8 ,input_length=2278 )(deep_inputs)
embedding = layers.LSTM(100,dropout=0.5, recurrent_dropout=0.5)(embedding)
#embedding = layers.Dense(256, activation='sigmoid')(embedding)
#embedding = layers.Dropout(0.5)(embedding)
#embedding = layers.Flatten()(embedding)
embedding_out = layers.Dense(5, activation='softmax')(embedding)
deep_model_lstm = keras.Model(inputs=deep_inputs,outputs=embedding_out)
deep_model_lstm.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
deep_model_lstm.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 2278)              0         
_________________________________________________________________
embedding_3 (Embedding)      (None, 2278, 8)           80000     
_________________________________________________________________
lstm_3 (LSTM)                (None, 100)               43600     
_________________________________________________________________
dense_3 (Dense)              (None, 5)                 505       
Total params: 124,105
Trainable params: 124,105
Non-trainable params: 0
_________________________________________________________________


In [28]:
deep_model_lstm.fit(X_train_seq_trunc,y_train_oh,epochs=10,batch_size=512)

Epoch 1/10
1621/1621 [==============================] - 23s 14ms/step - loss: 0.3386 - acc: 0.8902
Epoch 2/10
1621/1621 [==============================] - 25s 15ms/step - loss: 0.3398 - acc: 0.8927
Epoch 3/10
1621/1621 [==============================] - 25s 15ms/step - loss: 0.2801 - acc: 0.9241
Epoch 4/10
1621/1621 [==============================] - 24s 15ms/step - loss: 0.3077 - acc: 0.9013
Epoch 5/10
1621/1621 [==============================] - 24s 15ms/step - loss: 0.2800 - acc: 0.9124
Epoch 6/10
1621/1621 [==============================] - 22s 14ms/step - loss: 0.3004 - acc: 0.9038
Epoch 7/10
1621/1621 [==============================] - 22s 14ms/step - loss: 0.2748 - acc: 0.9124
Epoch 8/10
1621/1621 [==============================] - 22s 14ms/step - loss: 0.2928 - acc: 0.9050
Epoch 9/10
1621/1621 [==============================] - 22s 14ms/step - loss: 0.2952 - acc: 0.9062
Epoch 10/10
1621/1621 [==============================] - 24s 15ms/step - loss: 0.2521 - acc: 0.9266


In [70]:
deep_model_lstm.evaluate(X_test_seq_trunc,y_test_oh)

406/406 [==============================] - 6s 14ms/step


[1.6093975376025798, 0.19211822682119942]

## Model selection

### Model       Result 
1.   NN     = >  97.04%
2.   CNN    =>   96.41%
3.   GRU    =>   76.85%
4.   LSTM   =>   73%
5.   Bidire =>


we can try other model and evaluate model but due to RAM and processor is not that much effifcient of my system , it will take lot of time to train the model.

So we will finalize NN to go further to create API for this model.
again we can optimize the result using hyperparameter tuning but I guess we have already achieved good result and for increasing it for 1 2 % we dont need to add much lines of code.

